In [19]:
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from plotnine import ggplot, aes, geom_point, geom_line
from plotnine.themes import theme_minimal



In [113]:
# import data
data=pandas.read_csv('refugee_data/refugee_data/collected_data.csv')

In [114]:
data=data[data["country"]!="Russian Federation"]
data=data[data["touching"]==1]

In [115]:
data

,Unnamed: 0,country,individualPerCountry,conflict_start_year,bilateral_migration,export_trade_share,migrant_ratio,same_language,touching,Total_pop_left_conflict_zone,...,Afghanistan,Burundi,Central African Republic,Democratic Republic of the Congo,Nigeria,Somalia,South Sudan,Syria,Ukraine,Venezuela
0,1,Iran,38180,2001,0,1.730,3.2,0,1,2221828,...,1,0,0,0,0,0,0,0,0,0
1,2,Pakistan,117550,2001,14461,34.310,1.5,0,1,2221828,...,1,0,0,0,0,0,0,0,0,0
2,3,Tajikistan,5710,2001,1571,0.580,2.9,0,1,2221828,...,1,0,0,0,0,0,0,0,0,0
3,4,Uzbekistan,13020,2001,0,0.240,3.5,0,1,2221828,...,1,0,0,0,0,0,0,0,0,0
4,5,Democratic Republic of the Congo,40529,2015,0,11.010,1.1,1,1,263736,...,0,1,0,0,0,0,0,0,0,0
5,6,Rwanda,47989,2015,34616,1.410,4.3,1,1,263736,...,0,1,0,0,0,0,0,0,0,0
6,7,Tanzania,126663,2015,72272,1.750,2.8,1,1,263736,...,0,1,0,0,0,0,0,0,0,0
8,9,Cameroon,346784,2013,850,13.250,2.0,1,1,737973,...,0,0,1,0,0,0,0,0,0,0
9,10,Chad,121253,2013,9142,0.160,3.2,1,1,737973,...,0,0,1,0,0,0,0,0,0,0
10,11,Democratic Republic of the Congo,210499,2013,0,0.500,1.1,1,1,737973,...,0,0,1,0,0,0,0,0,0,0


In [116]:
# import data

dummies = data[['Afghanistan', 'Burundi',
       'Central African Republic', 'Democratic Republic of the Congo',
       'Nigeria', 'Somalia', 'South Sudan', 'Syria', 'Ukraine', 'Venezuela']]

conflict = pd.Series(dummies.columns[np.where(dummies!=0)[1]])

data['conflict'] = conflict

cols_to_scale = ['population','gdp_millions','bilateral_migration']

scaler = MinMaxScaler()
for col in cols_to_scale:
    print(col)
    normed = pd.DataFrame()
    for y, x in data.groupby('conflict'):
        norm_ = [i[0] for i in scaler.fit_transform(x[col].values.reshape(-1,1))]
        countries = x['country']
        res = pd.DataFrame(tuple(zip(countries,norm_)), columns=['country',f"{col}_norm"])
        normed = normed.append(res)
    data = pd.merge(data, normed, left_on='country', right_on='country', how='left')

ukr = data[data['conflict']=='Ukraine']

ukr[['country','conflict','population_norm', 'population', 
      'gdp_millions_norm','gdp_millions','bilateral_migration_norm','bilateral_migration']]

population
gdp_millions


<ipython-input-116-dabd08c28ef7>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<ipython-input-116-dabd08c28ef7>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<ipython-input-116-dabd08c28ef7>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<ipython-input-116-dabd08c28ef7>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<ipython-input-116-dabd08c28ef7>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<ipython-input-116-dabd08c28ef7>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pan

bilateral_migration


<ipython-input-116-dabd08c28ef7>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<ipython-input-116-dabd08c28ef7>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<ipython-input-116-dabd08c28ef7>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<ipython-input-116-dabd08c28ef7>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<ipython-input-116-dabd08c28ef7>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<ipython-input-116-dabd08c28ef7>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pan

,country,conflict,population_norm,population,gdp_millions_norm,gdp_millions,bilateral_migration_norm,bilateral_migration
8107,Ethiopia,Ukraine,1.0,117876227.0,1.000000,92757,1.0,0
8108,Ethiopia,Ukraine,1.0,117876227.0,1.000000,92757,1.0,0
8109,Ethiopia,Ukraine,1.0,117876227.0,1.000000,92757,1.0,0
8110,Ethiopia,Ukraine,1.0,117876227.0,1.000000,92757,1.0,0
8111,Ethiopia,Ukraine,1.0,117876227.0,1.000000,92757,1.0,0
...,...,...,...,...,...,...,...,...
8614,Uganda,Ukraine,0.0,42729036.0,0.099549,43243,0.0,0
8615,Uganda,Ukraine,0.0,42729036.0,0.099549,43243,0.0,0
8616,Uganda,Ukraine,0.0,42729036.0,0.099549,43243,0.0,0
8617,Uganda,Ukraine,0.0,42729036.0,0.099549,43243,0.0,0


In [117]:
data.head(100)

,Unnamed: 0,country,individualPerCountry,conflict_start_year,bilateral_migration,export_trade_share,migrant_ratio,same_language,touching,Total_pop_left_conflict_zone,...,Nigeria,Somalia,South Sudan,Syria,Ukraine,Venezuela,conflict,population_norm,gdp_millions_norm,bilateral_migration_norm
0,1,Iran,38180,2001,0,1.73,3.2,0,1,2221828,...,0,0,0,0,0,0,Afghanistan,0.338673,1.000000,0.000000
1,2,Pakistan,117550,2001,14461,34.31,1.5,0,1,2221828,...,0,0,0,0,0,0,Afghanistan,1.000000,0.236273,1.000000
2,3,Tajikistan,5710,2001,1571,0.58,2.9,0,1,2221828,...,0,0,0,0,0,0,Afghanistan,0.000000,0.000000,0.108637
3,4,Uzbekistan,13020,2001,0,0.24,3.5,0,1,2221828,...,0,0,0,0,0,0,Afghanistan,0.118472,0.053439,0.000000
4,5,Democratic Republic of the Congo,40529,2015,0,11.01,1.1,1,1,263736,...,0,0,0,0,0,0,Burundi,1.000000,0.755179,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5,Democratic Republic of the Congo,40529,2015,0,11.01,1.1,1,1,263736,...,0,0,0,0,0,0,Burundi,1.000000,0.755179,0.000000
96,5,Democratic Republic of the Congo,40529,2015,0,11.01,1.1,1,1,263736,...,0,0,0,0,0,0,Burundi,1.000000,0.755179,0.000000
97,5,Democratic Republic of the Congo,40529,2015,0,11.01,1.1,1,1,263736,...,0,0,0,0,0,0,Burundi,1.000000,0.755179,0.000000
98,5,Democratic Republic of the Congo,40529,2015,0,11.01,1.1,1,1,263736,...,0,0,0,0,0,0,Burundi,1.000000,0.755179,0.000000


In [102]:
#train model without Ukraine so we use withoutUkrain
withoutUkrainData=data[data["Ukraine"]!=1]
withoutUkrainData=data[data["touching"]==1]


In [103]:
# set y variable we are trying to predict
y=withoutUkrainData['percent_Refugees_of_Conflict']

In [104]:
# set independant variables 
features_normalized = withoutUkrainData[[
    "Afghanistan",
        "Nigeria",
    "Burundi",
    "Central African Republic",
    "Democratic Republic of the Congo",
    "Somalia",
    "South Sudan",
    "Syria",
    "Venezuela" ,
    "v2xeg_eqdr",
    "v2x_libdem",
    "population_norm",
"gdp_millions_norm",
"bilateral_migration_norm"
]]

###   We also tried to use these additional features in the model, however none of these features had a high enough p value to be included in the final model
 1. "normalized_qrdp"
 2. "same_language"
 3. "touching"
 4. "normalized_lib"
 5. "normalized_pop"
 6. "normalized_migr_ratio"
 7. "normalized_gdp"
 8. "normalized_export_trade"
 9. "normalized_remittances"
 

In [28]:
# Run the linear regression. 
import statsmodels.api as sm

In [105]:
results=sm.OLS(y,features_normalized).fit()

In [106]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      OLS Regression Results                                     
=================================================================================================
Dep. Variable:     percent_Refugees_of_Conflict   R-squared (uncentered):                   0.955
Model:                                      OLS   Adj. R-squared (uncentered):              0.955
Method:                           Least Squares   F-statistic:                          6.912e+04
Date:                          Thu, 14 Apr 2022   Prob (F-statistic):                        0.00
Time:                                  19:38:37   Log-Likelihood:                          49078.
No. Observations:                         45518   AIC:                                 -9.813e+04
Df Residuals:                             45504   BIC:                                 -9.800e+04
Df Model:                                    14                                                  
Covariance Type:                      nonrobust                                                  
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Afghanistan                          0.3826      0.041      9.271      0.000       0.302       0.464
Nigeria                              0.4965      0.008     65.083      0.000       0.482       0.511
Burundi                              0.2623      0.003     96.842      0.000       0.257       0.268
Central African Republic             0.3125      0.002    125.292      0.000       0.308       0.317
Democratic Republic of the Congo     0.2930      0.003    114.705      0.000       0.288       0.298
Somalia                              0.4119      0.004    105.377      0.000       0.404       0.420
South Sudan                          0.2108      0.003     78.635      0.000       0.206       0.216
Syria                                0.3229      0.041      7.816      0.000       0.242       0.404
Venezuela                           -0.4315      0.048     -9.015      0.000      -0.525      -0.338
v2xeg_eqdr                          -0.9871      0.007   -142.363      0.000      -1.001      -0.974
v2x_libdem                           1.9944      0.012    161.967      0.000       1.970       2.019
population_norm                     -0.0718      0.001    -49.135      0.000      -0.075      -0.069
gdp_millions_norm                   -0.1026      0.001    -70.188      0.000      -0.105      -0.100
bilateral_migration_norm             0.0564      0.003     18.887      0.000       0.051       0.062
==============================================================================
Omnibus:                    13657.468   Durbin-Watson:                   0.033
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           127673.197
Skew:                          -1.175   Prob(JB):                         0.00
Kurtosis:                      10.861   Cond. No.                         120.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [107]:
features_to_predict=data[[
   "Afghanistan",
        "Nigeria",
    "Burundi",
    "Central African Republic",
    "Democratic Republic of the Congo",
    "Somalia",
    "South Sudan",
    "Syria",
    "Venezuela" ,
    "v2xeg_eqdr",
    "v2x_libdem",
    "population_norm",
"gdp_millions_norm",
"bilateral_migration_norm"

]]
shares = results.predict(features_to_predict)
data['predicted_shares'] = shares
ukr_results = data[data['Ukraine']==1][['country','percent_Refugees_of_Conflict','predicted_shares']]
ukr_results.to_csv('ukraine_model_results.csv',index=False)

In [108]:
ukr_results

,country,percent_Refugees_of_Conflict,predicted_shares
80341,Hungary,0.093523,0.049938
80342,Moldova,0.092987,0.573827
80343,Poland,0.582061,-0.069171
80344,Romania,0.153415,0.674038
80345,Slovakia,0.070598,0.739518


# Retrain model with Ukraine Conflict data included

In [76]:
y=data['percent_Refugees_of_Conflict']
features_normalized = data[[
   "Afghanistan",
        "Nigeria",
    "Burundi",
    "Central African Republic",
    "Democratic Republic of the Congo",
    "Somalia",
    "South Sudan",
    "Syria",
    "Venezuela" ,
    "v2xeg_eqdr",
    "v2x_libdem",
    "population_norm",
"gdp_millions_norm",
"bilateral_migration_norm"
    

]]

In [72]:
y=data['percent_Refugees_of_Conflict']
features_normalized = data[[
   "Afghanistan",
        "Nigeria",
    "Burundi",
    "Central African Republic",
    "Democratic Republic of the Congo",
    "Somalia",
    "South Sudan",
    "Syria",
    "Venezuela" ,
    "v2xeg_eqdr",
    "v2x_libdem",
    "population_norm",
"gdp_millions_norm",
"bilateral_migration_norm"
    

]]

In [77]:
results=sm.OLS(y,features_normalized).fit()

In [78]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      OLS Regression Results                                     
=================================================================================================
Dep. Variable:     percent_Refugees_of_Conflict   R-squared (uncentered):                   0.865
Model:                                      OLS   Adj. R-squared (uncentered):              0.865
Method:                           Least Squares   F-statistic:                          3.705e+04
Date:                          Thu, 14 Apr 2022   Prob (F-statistic):                        0.00
Time:                                  19:31:32   Log-Likelihood:                          61968.
No. Observations:                         80742   AIC:                                 -1.239e+05
Df Residuals:                             80728   BIC:                                 -1.238e+05
Df Model:                                    14                                                  
Covariance Type:                      nonrobust                                                  
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Afghanistan                          0.2846      0.056      5.059      0.000       0.174       0.395
Nigeria                              0.3199      0.007     43.602      0.000       0.305       0.334
Burundi                              0.1876      0.002     75.250      0.000       0.183       0.192
Central African Republic             0.3216      0.003    116.708      0.000       0.316       0.327
Democratic Republic of the Congo     0.4008      0.003    150.605      0.000       0.396       0.406
Somalia                              0.1656      0.003     60.167      0.000       0.160       0.171
South Sudan                          0.3490      0.003    133.540      0.000       0.344       0.354
Syria                                0.3510      0.050      6.974      0.000       0.252       0.450
Venezuela                           -0.0343      0.034     -0.999      0.318      -0.102       0.033
v2xeg_eqdr                          -0.6585      0.008    -77.731      0.000      -0.675      -0.642
v2x_libdem                           0.8817      0.014     63.615      0.000       0.855       0.909
population_norm                     -0.0317      0.001    -21.260      0.000      -0.035      -0.029
gdp_millions_norm                   -0.0392      0.002    -25.218      0.000      -0.042      -0.036
bilateral_migration_norm             0.0540      0.003     16.870      0.000       0.048       0.060
==============================================================================
Omnibus:                     7387.378   Durbin-Watson:                   0.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            53290.437
Skew:                           0.057   Prob(JB):                         0.00
Kurtosis:                       6.978   Cond. No.                         124.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [79]:
features_to_predict=data[[
         "Afghanistan",
        "Nigeria",
    "Burundi",
    "Central African Republic",
    "Democratic Republic of the Congo",
    "Somalia",
    "South Sudan",
    "Syria",
    "Venezuela" ,
    "v2xeg_eqdr",
    "v2x_libdem",
    "population_norm",
"gdp_millions_norm",
"bilateral_migration_norm",]]
    

shares = results.predict(features_to_predict)
data['predicted_shares'] = shares
ukr_results = data[data['Ukraine']==1][['country','percent_Refugees_of_Conflict','predicted_shares']]
ukr_results.to_csv('ukraine_model_results.csv',index=False)

In [80]:
ukr_results

,country,percent_Refugees_of_Conflict,predicted_shares
80468,Hungary,0.093523,-0.107009
80469,Moldova,0.092987,0.085935
80470,Poland,0.582061,-0.171131
80471,Romania,0.153415,0.202144
80472,Russian Federation,0.081165,-0.358244
80473,Slovakia,0.070598,0.150724
